<a href="https://colab.research.google.com/github/marvinmouroum/End-to-End-EEG-Classifier/blob/master/CNN_simple_v1_2_Merging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A CNN to classify EEG data

##Packages

This section imports all the needed packages

In [0]:
!pip install pydrive

import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim


##### START OF ADDITION OF MY CODE

import numpy as np
import os.path
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import io
from googleapiclient.http import MediaIoBaseDownload

from scipy.io import loadmat
from pathlib import Path
import matplotlib.pyplot as plt
import scipy.io as sio
import sys
import pickle

## PyTorch 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import Tensor
import torch.optim as optim
import torchvision

import math #for calculus



     |████████████████████████████████| 993kB 6.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built pydrive


## Download the dataset

This section contains all the functions in order to download the dataset and clean it,.

In [0]:
#np.set_printoptions(threshold=sys.maxsize)

def load_file(file_id, content):
  
  print("loading file:\n",file_id)
  
  drive = GoogleDrive(gauth)
  downloaded = drive.CreateFile({'id': file_id})
  downloaded.GetContentFile(content)
  
  mat = sio.loadmat(content, squeeze_me=True, struct_as_record=False)

  print("loaded dataset  ...")

  o=mat['o']
  data=mat['o'].data
  labels=mat['o'].marker

  found =  []
  
  valid_data   = []
  valid_labels = []

  last_ind = 0
  
  print("\ndataset contains ", data.shape ," entries" )
  
  #check if there are invalid labels
  if np.amax(labels) <= 3:
    valid_data.append(data)
    valid_labels.append(labels)
  
  #cut out chunks if there were invalid labels
  else:
    
    #find the start
    start = np.where((labels == 99))
    clean_start = start[0][-1] + 1
    
    #find the breaks
    breaks = np.where(labels == 91)
    
    #the very first break
    if breaks[0].size != 0:
      clean_breaks = [breaks[0][0]]
    
      #the intermediate break points
      for i in range(len(breaks[0])-1):
        if breaks[0][i] + 1 != breaks[0][i+1]:
          clean_breaks.append(breaks[0][i])
          clean_breaks.append(breaks[0][i+1])
    
      #the very last break point
      if clean_breaks[-1] != breaks[-1][-1]:
        clean_breaks.append(breaks[-1][-1])
        
      valid_data.append(data[clean_start:clean_breaks[0]])
      valid_labels.append(labels[clean_start:clean_breaks[0]])
      
      for b in range(1,len(clean_breaks)-1,2):
        valid_data.append(data[clean_breaks[b]+1:clean_breaks[b+1]])
        valid_labels.append(labels[clean_breaks[b]+1:clean_breaks[b+1]])
    else:
      print("\n !!! DATASET does NOT contain BREAK flags !!!")
    
    #find the end
    finish = np.where(labels == 92)
    if finish[0].size != 0:
      clean_finish = finish[0][0]
    else:
      clean_finish = labels.shape[0]
        
    
    if breaks[0].size != 0:
      valid_data.append(data[clean_breaks[-1]+1:clean_finish])
      valid_labels.append(labels[clean_breaks[-1]+1:clean_finish])
      
    else:
      valid_data.append(data[clean_start:clean_finish])
      valid_labels.append(labels[clean_start:clean_finish])
    

  print("cleaned data  ...")
  
  
  newdata  = np.vstack(valid_data[0:len(valid_data)])
  newlabels = np.hstack(valid_labels[0:len(valid_labels)])
  
  return (newdata,newlabels)

In [0]:
def combine_data(data,labels):
  return np.hstack((data,labels.reshape((labels.shape[0],1))))



##Video Stream Creation

The imported data is converted into a 3D image. The electrodes are reconstructed assuming the head is a 3 dimensional ellipse.
The calculation can be seen at:

https://docs.google.com/spreadsheets/d/1TetDbE-HkAQGnVVIFbPGI4ugbktgdic_TFtsFyWEp0w/edit?usp=sharing

The created files can be stored on the drive.

In [0]:
## video maker and data extractor

pos=np.zeros((22,3), dtype=int)

pos[0]=-2,10,0
pos[1]=2,10,0
pos[3]=-4,7,3
pos[4]=4,7,3
pos[5]=-5,0,4
pos[6]=5,0,4
pos[7]=-4,-7,3
pos[8]=4,-7,3
pos[9]=-2,-10,0
pos[10]=2,-10,0
pos[11]=-7,0,-5
pos[12]=7,0,5
pos[13]=-6,6,0
pos[14]=6,6,0
pos[15]=-7,0,0
pos[16]=7,0,0
pos[17]=-6,-6,0
pos[18]=6,-6,0
pos[19]=0,7,4
pos[20]=0,0,6
pos[21]=0,-7,4

def make_3d_point(x,y,z,r,theta,phi):
  return [
      x + int(r * math.cos(theta)*math.sin(phi)),
      y + int(r * math.sin(theta)*math.sin(phi)),
      z + int(r * math.cos(phi))
         ]

def make_3d_data(image3d_data):
  #creating the matrix
  matrix = np.zeros([12,16,22]) # z,x,y
  
  #looping through data
  for point in image3d_data:
    
    # creating the indice by shifting the coordinate system from the center to left corner
    newpoint = [int(point[0]) + 7,int(point[1]) + 10,int(point[2]) + 5]
    
    if newpoint[0] >= matrix.shape[1] or newpoint[1] >= matrix.shape[2] or newpoint[2] >= matrix.shape[0] or newpoint[0] < 0 or newpoint[1] < 0 or newpoint[2] < 0:
      #print("did not add point")
      #print(newpoint)
      #print(matrix.shape)
      continue

    
    #assigning the value
    if abs(matrix[newpoint[2],newpoint[0],newpoint[1]]) <= abs(point[3]):
        matrix[newpoint[2],newpoint[0],newpoint[1]] = int(point[3])

    
    #creating variables for creating a virtual sphere around poi
    rho = [2,3] # distance (in pixels) that we are interpolating from poi
    #theta  angle in radians    [0,2pi]
    #phi    angle in radians    [0, pi]
    
    #creating new interpolated points
    for r in rho:
      for theta in range(0,360,45):
        for phi in range(0,180,45):
          p = make_3d_point(newpoint[0],newpoint[1],newpoint[2],r,math.radians(theta),math.radians(phi))
          
          #checking if its in bounds
          if p[0] < matrix.shape[1] and p[1] < matrix.shape[2] and p[2] < matrix.shape[0]:
            if abs(matrix[p[2],p[0],p[1]]) <= abs(point[3]/r):
                matrix[p[2],p[0],p[1]] = int(point[3]/r)
                     
  return matrix

 
def get_data(data,batch_size=256, test_batch_size=256):
  

  # Create train and validation splits
  num_samples = len(data)
  training_samples = int(num_samples*0.5+1)
  validation_samples = num_samples - training_samples

  #training_data, validation_data = torch.utils.data.random_split(data, [training_samples, validation_samples])
  training_data   = data[0:training_samples]
  validation_data = data[training_samples:-1]

  # Initialize dataloaders
  train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=False)
  val_loader = torch.utils.data.DataLoader(validation_data, batch_size, shuffle=False)
  #test_loader = torch.utils.data.DataLoader(test_data, test_batch_size, shuffle=False)
  
  print("loading data")
  
  return train_loader, val_loader #, test_loader



def get_video_data(loader):
  
  #print("get videos:")

  batchsize = loader.shape[0]
  
  training_data   = np.empty([batchsize,22,4])
  
  train_label = np.empty((batchsize))
  
 # print("start collecting training data")
  
  for it,timestep_data in enumerate(loader):
    train3d = np.hstack((pos,timestep_data[0:22].reshape([22,1])))
    training_data[it] = train3d
    train_label[it] = timestep_data[22] 
 
    
  return (training_data, train_label)
  
  
  
def make_video(video_data):
  return make_3d_data(video_data)
  
  
  
def get_training_data(train,mean,std):

  stream = np.zeros([train[0].shape[0],1,12,16,22])
  
  for i,image in enumerate(train[0]):
    if i >= stream.shape[0]:
      break
    stream[i,0] = make_video(image)
  
  tensor = torch.from_numpy(stream)

  tensor = (tensor - mean) / std
  
  return (tensor,train[1])

def save_video(data,batch_size=256,path='VideoBatches/',mean=0,std=1):
  
  print("saving video files in VideoBatches with batch size: ",batch_size)
  
  train_loader, val_loader = get_data(data,batch_size)
  
  print("\nsaving training data\n")
  
  for batch_idx, inputs in enumerate(train_loader):
      
    if batch_idx % 100 == 0:
      print("saving batches: " , batch_idx, "-", batch_idx+100)
      
    result_data = get_video_data(inputs)
    result = get_training_data(result_data,mean,std)
    
    name = path + "train_" + str(batch_idx) + ".pickle"
    
    with open(name, 'wb') as f:
      pickle.dump([result[0], result[1]], f)
      
  print("\nsaving validation data\n")
  
  for batch_idx, inputs in enumerate(train_loader):

    if batch_idx % 100 == 0:
      print("saving batches: " , batch_idx, "-", batch_idx+100)
      
    result_data = get_video_data(inputs)
    result = get_training_data(result_data,mean,std)
    
    name = path + "val_" + str(batch_idx) + ".pickle"
    
    with open(name, 'wb') as f:
      pickle.dump([result[0], result[1]], f)
    
    
    
  
  



In [0]:
from google.colab import drive as gdrive

gdrive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/DeepLearning/VideoBatches/subject01/'

def save_video_to_drive(_id,file_id,file_name,directory,batch_size=256):
  
  root_path = 'gdrive/My\ Drive/' + directory
  
  save_dir = root_path + 'subject' + str(_id)
  
  print("creating directory\n",save_dir)
  
  !mkdir $save_dir
  
  root_path = 'gdrive/My Drive/' + directory
  
  save_dir = root_path + 'subject' + str(_id)
  
  _dir = save_dir + '/'
  
  print("creating data for " , file_name)
  
  cleaned_data = load_file(file_id,file_name)
  
  mean = cleaned_data[0].mean()
  std  = cleaned_data[0].std() 
  
  data = combine_data(cleaned_data[0],cleaned_data[1])
  
  save_video(data,batch_size,_dir,mean,std)
  
  print("Done")
  


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!ls

file_ids = ['1FrXb6rTyqpE5SmNtP8HhygDmFf9Lw896','19cjwiSqdKK-bioqQK_vbptbYSYJGuop1','10zeWbMn_j8pl_sZrlzvNWSumedr2etca','1TqewoCjjRXZpEIxL_23ZQ8MP4L0RofZx','1-2pj-scr75Y3iI_gHUS17WXHpgQzT6_f','167lgnY18GUIJiy9mIungxtSk-LSTOoar','1v8RaiTYN82BUJuOXIeJH5YvUWP42ZyNc','15AdOQt4rNEPe2WKjVfqwOJluCQHQCW2E','1bIDO5pDIvs5KRNmew-weFCN8rCaHU8v5','1WU2U86bPxXCjc-laHca1jedq_PqIDKiL','1hKxhB6paBy_MZVMGlcmWFFF8XO3D_1m4','1U7iQryUNXfVhls6tjDz81yK7UrAUTKGB','16OHZ3foYB8JKvC1TcqO6gYvwPBnYqpci','11VRncgCXRjBktla-cNOE2W-w8stywB6u','1YMq3v0FmWwWB519dB8AvybeXkHPcHMFF','1ilT5ZXa_6rpGf7pb02rX0-PuMbuSzIlI','1pDy1AI4Z9mLRJ6gBkzHBoc7qI3TKMErP']

number = len(file_ids)

for i in range(6,int(number/2)):
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  save_video_to_drive(str(i),file_ids[i],'train'+ str(i)+ '.mat','DeepLearning/VideoBatches/',256)

adc.json  gdrive  sample_data  train7.mat
creating directory
 gdrive/My\ Drive/DeepLearning/VideoBatches/subject6
mkdir: cannot create directory ‘gdrive/My Drive/DeepLearning/VideoBatches/subject6’: File exists
creating data for  train6.mat
loading file:
 1v8RaiTYN82BUJuOXIeJH5YvUWP42ZyNc
loaded dataset  ...

dataset contains  (667600, 22)  entries
cleaned data  ...
saving video files in VideoBatches with batch size:  256
loading data

saving training data

saving batches:  0 - 100
saving batches:  100 - 200
saving batches:  200 - 300
saving batches:  300 - 400
saving batches:  400 - 500
saving batches:  500 - 600
saving batches:  600 - 700
saving batches:  700 - 800
saving batches:  800 - 900
saving batches:  900 - 1000
saving batches:  1000 - 1100
saving batches:  1100 - 1200
saving batches:  1200 - 1300
saving batches:  1300 - 1400

saving validation data

saving batches:  0 - 100
saving batches:  100 - 200
saving batches:  200 - 300
saving batches:  300 - 400
saving batches:  400 -

## Visualize 3D Images

This section contains the functions in order create a 3d scatter plot and save it on the drive

In [0]:
      
def make_plots(name,start=0,end=1):
  
  with open(name, 'rb') as f:
    result = pickle.load(f)
    
  points = []
  cocos  = []

  for i in range(start,end):

    point = np.zeros([1,3])
    coco = np.chararray([1])

    coco[0] = '0'

    for z in range(result[0][i,0].shape[0]):
      for x in range(result[0][i,0,z].shape[0]):
        for y in range(result[0][i,0,z,x].shape[0]):
          if abs(result[0][i,0,z,x,y]) > 0:
            raw   = result[0][i,0,z,x,y]
            clean = (raw + 1)/2
            point = np.append(point,[[x,y,z]],axis=0)
            coco = np.append(coco,['%.5f' % clean], axis=0)
        

    coco = coco[1:coco.shape[0]].reshape([coco.shape[0]-1])
    point = point[1:point.shape[0]].reshape([point.shape[0]-1,3])
    
    points.append(point)
    cocos.append(coco)

    
    
    
  return (points, cocos, result[1])

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

import sys
np.set_printoptions(threshold=sys.maxsize)


def save_3d_plots(save_to,data_path,start,end,plot=False):

  video = make_plots(data_path,start,end)


  for i, frame in enumerate(video[0]):
    print("frame ",i)
  
    point = frame
    coco  = video[1][i]

    data = np.empty([point.shape[0],3,1])
  
    data[:,0,0] = point[:,0]
    data[:,1,0] = point[:,1]
    data[:,2,0] = point[:,2]

    colors = coco

    groups = ("coffee", "tea", "water") 

    # Create plot
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax = fig.gca(projection='3d')

    print("label is -> " , video[2][i])
    
    if video[2][i] == 1:
      ax.set_facecolor('xkcd:mint green')
    elif video[2][i] == 2:
      ax.set_facecolor('xkcd:salmon')
    elif video[2][i] == 3:
      ax.set_facecolor('xkcd:blue')

    for data, color in zip(data, colors):
      x, y, z = data
      ax.scatter(x, y, z, alpha=0.8, c=color, edgecolors='none', s=30)

    plt.title('Matplot 3d scatter plot')
    plt.legend(loc=2)
    
    if plot == True:
      plt.show()
  
    image_name = save_to + "brain_" + str(start + i) + ".jpg"
    plt.savefig(image_name)
    
    if plot == False:
      plt.close(fig)

In [0]:
root_path = 'gdrive/My Drive/DeepLearning/'

path = root_path + "VideoBatches/subject0/train_1178.pickle"
save = root_path + "BrainImage/"

!mkdir $save

start = 0
end   = 254

save_3d_plots(save,path,start,end,plot=False)

!ls

## CNN

To-Do: 
- Find a good architecture
- Test with more data

In [0]:
from google.colab import drive as gdrive

gdrive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/DeepLearning/VideoBatches/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls

gdrive	sample_data


In [0]:
class SimpleLSTM(nn.Module):
    def __init__(self, input_size=22, hidden_size=22, output_size=4,
                 device='cuda:0'):
        super(SimpleLSTM, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        
        self.i2h = nn.LSTM(input_size, 22,hidden_size,bias=False,batch_first=False)
        self.i2o = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden=None, cell=None, device='cuda:0'):
        outputs = []
        
        if hidden==None:
          #print("hidden input hidden:",input.shape[1])
          hidden = self.init_hidden(input.shape[1]) 
          hidden.to(device)
        
        if cell==None:
          #print("hidden input cell:",input.shape[1])
          cell = self.init_hidden(input.shape[1])
          cell.to(device)
        
        output, (_,_)= self.i2h(input, (hidden,cell))
        
        output = self.i2o(output)
        
        return F.softmax(output,dim=2)

    def init_hidden(self,shape=1, device='cuda:0'):
        return torch.zeros(22, shape, self.hidden_size).to(device)
      
    def init_cell(self,shape=1, device='cuda:0'):
        return torch.zeros(22, shape, self.hidden_size).to(device)

In [0]:
class eeg_shallow_CNN(nn.Module):
    
    def __init__(self,name='shallow_network'):
        super(eeg_shallow_CNN, self).__init__()
        self.T = 120
        
        self.name = name
        
        self.training_epochs = 0
        
        self.lr_history       = []
        self.accuracy_history = []
        self.loss_history     = []
        
        self.conv1 = nn.Conv3d(1,6,(2,2,2), stride=(1,1,1),padding=(2,2,2),dilation=1)
        self.batchnorm1 = nn.BatchNorm3d(6)
        
        self.conv2 = nn.Conv3d(6,12,(2,2,2), stride=(1,1,1),padding=(1,1,1),dilation=1)
        self.batchnorm2 = nn.BatchNorm3d(12)
        
        self.pooling1 = nn.MaxPool3d((2,2,2), stride=(1,1,1), dilation=1)
        
        #layer after max pool
        self.conv3 = nn.Conv3d(12,17,(2,2,2), stride=(1,1,1),padding=(1,1,0),dilation=1)
        self.batchnorm3 = nn.BatchNorm3d(17)
        
        self.conv4 = nn.Conv3d(17,22,(1,1,1), stride=(1,1,1),dilation=1)
        self.batchnorm4 = nn.BatchNorm3d(22)
        
        self.pooling2 = nn.MaxPool3d((2,3,2), stride=(1,1,1), dilation=1)
        
        #Layer after next max pool 
        self.conv5 = nn.Conv3d(22,11,(3,4,4), stride=(2,2,1),padding=(2,2,2),dilation=1)
        self.batchnorm5 = nn.BatchNorm3d(11)
        self.conv6 = nn.Conv3d(11,1,(3,4,1), stride=(2,2,1),padding=(0,1,0),dilation=1)
        self.batchnorm6 = nn.BatchNorm3d(1)
        
        self.pooling3 = nn.MaxPool3d((4,5,3), stride=(2,1,1), dilation=1)
        
        #fully connected at the end
        self.fc1 = torch.nn.Linear(in_features=1 * 1 * 22, out_features=11)
        self.fc2 = torch.nn.Linear(in_features=11, out_features=4)
        
        self.rnn = SimpleLSTM()
        
        # robust weight initialization
        torch.nn.init.xavier_normal_(self.conv1.weight)
        torch.nn.init.xavier_normal_(self.conv2.weight)
        torch.nn.init.xavier_normal_(self.conv3.weight)
        torch.nn.init.xavier_normal_(self.conv4.weight)
        torch.nn.init.xavier_normal_(self.conv5.weight)
        torch.nn.init.xavier_normal_(self.conv6.weight)

        torch.nn.init.xavier_normal_(self.fc1.weight)
        torch.nn.init.xavier_normal_(self.fc2.weight)
      

        
    def forward(self, x):

        # first set of CNNs and then a max pool
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = F.relu(x)
        
        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = F.relu(x)
        
        x = self.pooling1(x)
        
        # second set
        
        x = self.conv3(x)
        x = self.batchnorm3(x)
        x = F.relu(x)
        
        x = self.conv4(x)
        x = self.batchnorm4(x)
        x = F.relu(x)
        
        x = self.pooling2(x)
        
        # set 3
        
        x = self.conv5(x)
        x = self.batchnorm5(x)
        x = F.relu(x)
        
        x = self.conv6(x)
        x = self.batchnorm6(x)
        x = F.relu(x)
        
        x = self.pooling3(x)
        
        batch_size, timesteps, C, H, W = x.size()
        
        x = x.view(batch_size, timesteps, W)
        
        return self.rnn(x)
      
    def save(self,root='gdrive/My\ Drive/DeepLearning/Nets'):
      !mkdir $root
  
      path = root.replace('\\','') + '/' + self.name + '.pickle'
    
      print("\nsaving network under:\n",path)
      
      with open(path, 'wb') as f:
        pickle.dump(self, f)
        
    def next_epoch(self,lr,acc_hist,loss_hist):
      self.training_epochs += 1
      self.lr_history.append(lr)
      self.accuracy_history.append(acc_hist)
      self.loss_history.append(loss_hist)
    
    def reset(self):
      self.training_epochs = 0
      self.lr_history = []
      self.accuracy_history = []
      self.loss_history = []

In [0]:
def get_cost_function():
    
  cost_function = torch.nn.CrossEntropyLoss()
  return cost_function
    
def get_optimizer(net, lr, wd, momentum):
  #optimizer = torch.optim.SGD(net.parameters(), lr=lr, weight_decay=wd, momentum=momentum)
  optimizer = torch.optim.Adam(net.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=wd, amsgrad=False)
  return optimizer

In [0]:
#the ids for the folders in drive containing the pre processed data for every subject
# subject 7 is not valid and missing
# makes the training process more controllable

training_folder_ids = ['1VX1qChrP0CsxumaAqZ66HUYZdgO65jre', #subject 0
                       '1rOHo2Wu_Rfng1-nYVWy5YdeWH1jlxwQ4', #subject 1
                       '188Wrvwk6M1KDB1ydg5FSWk3PBhnyrX5D', #subject 2
                       '1U4XYEHLvIrtLMSJxbb-lt6clpjHx7o4-', #subject 3
                       '1k0GTcLtTuZVGEPg0zW_QrL3o7HiLH7Sy', #subject 4
                       '10D64lIdH7kxlxHUU5h6BWf40a4sLOF07', #subject 5
                       '1brTnGtGzH5v10J_OfWJnL4D0l0i8hU9x', #subject 6
                       '',
                       '1ouD338EaxRTDqvK5m6LxgGQQK7iVAvwt', #subject 8 !!!
                       '1TG49iJEtNBxSlqxtkOqKOJ2xClDX8GYq', #subject 9
                       '13-RkhEErzwuuNvMKwQrlvLlJ97B-IQY5', #subject 10
                       '1gaoPwQqketkoh6tk_PuxoEK1x5TzC8wp', #subject 11
                       '1IXOlZY_lAkc9p_BYho7bo5cu82nvsGOp', #subject 12
                       '13o2xxTWU56rPF-cTMsmOrWlCisrCgcPe', #subject 13
                       '1cASt5iWOe_GV0y5TaV7K7o6IzJqNkeHl', #subject 14
                       '12i7kMF7vJQvaZ37MD6bCm6Rui8OW203Y', #subject 15
                       '1TclA0tmxgnEhy8pquMYfPwWeEmFWiB7U'] #subject 16

In [0]:


from pydrive.drive import GoogleDrive

def get_preprocessed_data_count_from(subject):
  
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

  file_list = drive.ListFile({'q': "'" + training_folder_ids[subject] + "' in parents and trashed=false"}).GetList()

  return len(file_list)

#get_preprocessed_data_count_from(1)

In [0]:
def test(net, data_type, cost_function, device='cuda:0', start=0,end=16):
  samples = 0.
  sequences = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.
  cumulative_seq_accuracy = 0.

  seq = 200

  net.eval() # Strictly needed if network contains layers which has different behaviours between train and test
  
  with torch.no_grad():
    
    for subject in range(start,end,1):
      if subject == 7:
        continue
    
      print("\nTesting on subject ", subject)
      
      length = int(get_preprocessed_data_count_from(subject)/2)
    
      skipped = 0
      kept    = 0
    
      for i in range(length-1-4):
        
        if i % 300 == 0:
          print("on test batch: " , i, "-", i+300)
       
        results = []
        
        tmp_skipped = 0
        j = 0
        
        while j < 4 + tmp_skipped and i + 4 + tmp_skipped < length:
          j += 1
          
          name = root_path + "subject" + str(subject) + "/" + data_type + "_" + str(i+j) + ".pickle"
      
          with open(name, 'rb') as f:
            subresult = pickle.load(f)

          clean_sub = clean_seq(subresult[0],subresult[1],seq)
          
          if clean_sub == None:
            skipped += 1
            tmp_skipped += 1
            #print("skipped a sequence")
            continue
          else:
            kept += 1
            
          
          sub = (clean_sub[0],clean_sub[1])
          results.append(sub)
          
          
 
        if len(results) < 4:
          print("not enough data left for rnn -> next epoch pls")
          continue
        
        
        t_data   = Variable( torch.from_numpy(
                                         np.concatenate( 
                                        (np.concatenate((results[0][0],results[1][0]),  axis=0) , 
                                         np.concatenate((results[2][0],results[3][0]),  axis=0)) ,
                                        axis=0)
                              ),
                            requires_grad=True
                                   )
        
        t_labels = np.concatenate((np.concatenate((results[0][1],results[1][1]), axis=None) , 
                                   np.concatenate((results[2][1],results[3][1]), axis=None)) ,
                                  axis = None)
                 
        #print(t_data)
        
        if t_data.shape[0] < 4*seq:
          print("sequence length too short")
          print(t_data.shape)
          continue
        
        max_v = t_labels.max()
      
        if max_v > 3:
          #print("\nlabels were not correct" , max_v , ", skipped batch: " , i)
          continue
      
        #zero mean and normalize the data
        mean = t_data.mean()
        std  = t_data.std()
      
        t_data = (t_data - mean)/std
      
        # Load data into GPU
        inputs = t_data.float().to(device)
    
        targets = torch.LongTensor(t_labels.reshape((4,seq))[0:4,seq-1]).to(device)
        
        # Forward pass
        #outputs = net(inputs).reshape([inputs.shape[0],4])
        outputs = net(inputs).reshape([4,seq,4])[0:4,seq-1]
      
      
        # Apply the loss
        loss = cost_function(outputs, targets)


        #print("loss",loss)

        samples+=4
        
        sequences+=4
        
        cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
        
        _, predicted = outputs.max(1)
      
        #print("outputs from rnn -> ", rnnOutputs.shape)
        
        #seq_results = outputs.view(4,seq,outputs.shape[1])[0:4 , seq-1] # (4,4)
        
        #print("last outputs ->", seq_results, seq_results.shape)
        
        #_, predicted_s = seq_results.max(1)
        
        #print("max predicted value",predicted_s)
        
        #seq_targets   = targets.view(4,seq)[0:4,seq-1]
        
        #print("seq_targets -> ", seq_targets)
        
        #cumulative_seq_accuracy += predicted_s.eq(seq_targets).sum().item()
        
        cumulative_accuracy += predicted.eq(targets).sum().item()
      
      print("Accuracy after subject evaluation -> " , cumulative_accuracy/samples*100)
      #print("Sequence Accuracy after subject evaluation -> " , cumulative_seq_accuracy/sequences*100)
      
      print( int(100*skipped/(skipped+kept)) ,"% of the data was ignored due to sequence cleaning")
  
  
  return cumulative_loss/samples, cumulative_accuracy/samples*100, cumulative_accuracy/samples*100# cumulative_seq_accuracy/sequences*100


def train(net,data_type,optimizer,cost_function, device='cuda:0', start=0,end=16):
  samples = 0.
  sequences = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.
  cumulative_seq_accuracy = 0.

  seq = 200
  
  net.train() # Strictly needed if network contains layers which has different behaviours between train and test
  
  for subject in range(start,end,1):
    if subject == 7:
      continue
    
    print("\nTraining on subject ", subject)
      
    length = int(get_preprocessed_data_count_from(subject)/2)
    
    skipped = 0
    kept    = 0
    
    for i in range(length-1-4):
        
      if i % 300 == 0:
        print("on train batch: " , i, "-", i+300)
       
      results = []
        
      tmp_skipped = 0
      j = 0
        
      while j < 4 + tmp_skipped and i + 4 + tmp_skipped < length:
        j += 1
          
        name = root_path + "subject" + str(subject) + "/" + data_type + "_" + str(i+j) + ".pickle"
      
        with open(name, 'rb') as f:
          subresult = pickle.load(f)

        clean_sub = clean_seq(subresult[0],subresult[1],seq)
          
        if clean_sub == None:
          skipped += 1
          tmp_skipped += 1
            #print("skipped a sequence")
          continue
        else:
          kept += 1
            
          
        sub = (clean_sub[0],clean_sub[1])
        results.append(sub)
          
          
 
      if len(results) < 4:
        print("not enough data left for rnn -> next epoch pls")
        continue
        
        
      t_data   = Variable( torch.from_numpy(
                                         np.concatenate( 
                                        (np.concatenate((results[0][0],results[1][0]),  axis=0) , 
                                         np.concatenate((results[2][0],results[3][0]),  axis=0)) ,
                                        axis=0)
                              ),
                            requires_grad=True
                                   )
        
      t_labels = np.concatenate((np.concatenate((results[0][1],results[1][1]), axis=None) , 
                                   np.concatenate((results[2][1],results[3][1]), axis=None)) ,
                                  axis = None)
                 
        #print(t_data)
        
      if t_data.shape[0] < 4*seq:
        print("sequence length too short")
        print(t_data.shape)
        continue
      
      max_v = t_labels.max()
      
      if max_v > 3:
        #print("\nlabels were not correct" , max_v , ", skipped batch: " , i)
        continue
      
      #zero mean and normalize the data
      mean = t_data.mean()
      std  = t_data.std()
      
      t_data = (t_data - mean)/std
      
      # Load data into GPU
      inputs = t_data.float().to(device)
    
      
      targets = torch.LongTensor(t_labels.reshape((4,seq))[0:4,seq-1]).to(device)
        
      # Forward pass
      #outputs = net(inputs).reshape([inputs.shape[0],4])
      outputs = net(inputs).reshape([4,seq,4])[0:4,seq-1]
      
      
      # Apply the loss
      loss = cost_function(outputs, targets)

      # Backward pass
      loss.backward()
    
      # Update parameters
      optimizer.step()
    
      # Reset the optimizer
      optimizer.zero_grad()

      samples+=4
        
      sequences+=4
        
      cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
        
      _, predicted = outputs.max(1)
      
      #print("outputs from rnn -> ", rnnOutputs.shape)
        
      #seq_results = outputs.view(4,seq,outputs.shape[1])[0:4 , seq-1] # (4,4)
        
      #print("last outputs ->", seq_results, seq_results.shape)
        
      #_, predicted_s = seq_results.max(1)
        
      #print("max predicted value",predicted_s)
        
      #seq_targets   = targets.view(4,seq)[0:4,seq-1]
        
      #print("seq_targets -> ", seq_targets)
        
      #cumulative_seq_accuracy += predicted_s.eq(seq_targets).sum().item()
        
      cumulative_accuracy += predicted.eq(targets).sum().item()
      
    print("Accuracy after subject evaluation -> " , cumulative_accuracy/samples*100)
    #print("Sequence Accuracy after subject evaluation -> " , cumulative_seq_accuracy/sequences*100)
      
    print( int(100*skipped/(skipped+kept)) ,"% of the data was ignored due to sequence cleaning")
  
  
  return cumulative_loss/samples, cumulative_accuracy/samples*100, cumulative_accuracy/samples*100# cumulative_seq_accuracy/sequences*100

In [0]:
def clean_seq(data,seq,seq_len=200):
  
  if np.amax(seq) == 0:
    #print("returned something")
    return [data[0:seq_len],seq[0:seq_len]]
  
  #size of sequence
  seq_size = seq.size
  
  #get the first snippet
  
  first_good = False
  last_good  = False
  i=0
  
  while not last_good:
  
    while not first_good:
      
      snippet =       seq[seq_size-seq_len-i:seq_size-i]
      data_snippet = data[seq_size-seq_len-i:seq_size-i]
  
      #first value
      first_value = snippet[0]
  
      if first_value.item() == 0:
        #print("first value is good",first_value)
        first_good = True
      elif i == 0:
        i = seq_size-seq_len
        #print("moved snippet 1")
      else:
        #print("couldn't make sequence")
        #print(snippet)
        return
  
  
    #last value
    last_value = snippet[-1]
  
    #max value
    max_value =  np.amax(snippet)
    #print("max is -> ",max_value)
    
    if max_value == last_value:
      #print("last value is good",last_value)
      last_good = True
    elif i == 0:
      first_good = False
      i = seq_size-seq_len
      #print("moved snippet 2")
    elif not last_good or not first_good:
      #print("couldn't make sequence")
      #print(snippet)
      return
    
  #print(snippet)
  
  return [data_snippet,snippet]

## Main

In [0]:
def get_net(path):
  with open(path, 'rb') as f:
    result = pickle.load(f)
  
  return result

In [0]:
def main(net= None,
         netname= 'net',
         batch_size=128, 
         device='cuda:0', 
         learning_rate=1e-5, 
         weight_decay=0.000001, 
         momentum=0.9, 
         epochs=25,
         start=0,
         end =16):
  
  print("make data loaders")
  
  train_loader = "train"
  val_loader   = "val"
  
  print("network to device")
  if net == None:
    net = eeg_CNN(netname).to(device)
  else:
    print("continue training on net")
    net.to(device)
    net.name = netname
  
  print("optimizer")
  optimizer = get_optimizer(net , learning_rate, weight_decay, momentum)
  
  print("cost function")
  cost_function = get_cost_function()

  #print('Before training:')
  #train_loss, train_accuracy, seq_train_accuracy = test(net, train_loader, cost_function, device,start,end)
  #val_loss, val_accuracy, seq_val_accuracy      = test(net, val_loader, cost_function, device,start,end)
  
  #print('\t Training loss {:.5f}, Training accuracy {:.2f}, Sequence Training accuracy {:.2f}'.format(train_loss, train_accuracy, seq_rain_accuracy))
  #print('\t Validation loss {:.5f}, Validation accuracy {:.2f}, Sequence Validation accuracy {:.2f}'.format(val_loss, val_accuracy, seq_val_accuracy))
  #print('-----------------------------------------------------')
  
  lr = learning_rate
  #loss = train_loss
  loss = 0.30160
  
  for e in range(epochs):
    train_loss, train_accuracy, seq_train_accuracy = train(net, train_loader, optimizer, cost_function,device,start,end)
    
    dloss = 1 - train_loss/loss
    
    loss = train_loss
    
    print("change in loss is:" , dloss)
    
    lr = learning_rate + abs(math.cos(e/2)*learning_rate*10)
    
    print("new lr ->", lr)
      
    optimizer = get_optimizer(net, lr, weight_decay, momentum)
    
    #val_loss, val_accuracy, seq_val_accuracy = test(net, val_loader, cost_function)
    print('Epoch: {:d}'.format(e+1))
    print('\t Training loss {:.5f}, Training accuracy {:.2f}, Sequence Training accuracy {:.2f}'.format(train_loss, train_accuracy, seq_train_accuracy))
    #print('\t Validation loss {:.5f}, Validation accuracy {:.2f}, Sequence Validation accuracy {:.2f}'.format(val_loss, val_accuracy, seq_val_accuracy))
    print('-----------------------------------------------------')
    
    net.next_epoch(lr,train_accuracy,train_loss)
    net.save()
    
    

  #print('After training:')
  #train_loss, train_accuracy, seq_train_accuracy = test(net, train_loader, cost_function, device,start,end)
  #val_loss, val_accuracy, seq_val_accuracy      = test(net, val_loader, cost_function, device,start,end)
  #test_loss, test_accuracy = test(net, test_loader, cost_function)

  #print('\t Training loss {:.5f}, Training accuracy {:.2f}, Sequence Training accuracy {:.2f}'.format(train_loss, train_accuracy, seq_rain_accuracy))
  #print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy, seq_val_accuracy))
  #print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  #print('-----------------------------------------------------')

In [0]:
#net = get_net('gdrive/My Drive/DeepLearning/Nets/11-layer-CNN-37.pickle')

#try a new one
#net = eeg_shallow_CNN('CNN-LSTM-1')
net = get_net('gdrive/My Drive/DeepLearning/Nets/CNN-LSTM-16.pickle')
#net.reset()

#print("restarting on net from epoch ", net.training_epochs)
#print("current accuracy -> ",net.accuracy_history[-1])

main(net,
         netname='CNN-LSTM-all',
         batch_size=128, 
         device='cuda:0', 
         learning_rate=5e-4, 
         weight_decay=1e-9, 
         momentum=0.9, 
         epochs=5,
         start=0,
         end  =16+1)

make data loaders
network to device
continue training on net
optimizer
cost function

Training on subject  0
on train batch:  0 - 300
